In [ ]:
pip install python-telegram-bot --upgrade

In [ ]:
import random
import nltk
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import logging
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [ ]:
# BOT_CONFIG = {
#     'intents': {
#         'hello': {
#             'examples': ['Привет!', 'Здарова', 'Хей-хей!!'],
#             'responses': ['Хай', 'Добрый вечер!', 'Здравствуйте!']
#         },
#         'bye': {
#             'examples': ['Пока', 'Увидимся!', 'Покеда'],
#             'responses': ['До свидания', 'Прощайте', 'Сайонара!']
#         }
#     }
# }
with open('/content/BOT_CONFIG.json', 'r') as f:
  BOT_CONFIG = json.load(f)

In [ ]:
with open('/content/BOT_CONFIG1.json', 'w') as f:
   json.dump(BOT_CONFIG, f, ensure_ascii=False, indent=3)

# День 1

In [ ]:
def clean(text):
  text = text.lower()
  cleaned_text = ''
  for ch in text:
    if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя ':
      cleaned_text = cleaned_text + ch
  return cleaned_text

def get_intent(text):
  for intent in BOT_CONFIG['intents'].keys():
    for example in BOT_CONFIG['intents'][intent]['examples']:
      w1 = clean(example)
      w2 = clean(text)
      if nltk.edit_distance(w1, w2) / max(len(w1), len(w2)) < 0.4:
        return intent
  return 'интент не найден'

# День 2

In [ ]:
X = []
y = []

for intent in BOT_CONFIG['intents'].keys():
    try:
        for example in BOT_CONFIG['intents'][intent]['examples']:
            X.append(example)
            y.append(intent)
    except:
        pass

len(X), len(y), len(set(y))

(841, 841, 243)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
len(X_train), len(X_test)

(672, 169)

In [ ]:
vectorizer = CountVectorizer(preprocessor=clean, analyzer='char', ngram_range=(2,3))
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

len(vectorizer.get_feature_names())

2476

In [ ]:
log_reg = LogisticRegression(C=0.2)
log_reg.fit(X_train_vect, y_train)
log_reg.score(X_train_vect, y_train)

0.8065476190476191

In [ ]:
log_reg.score(X_test_vect, y_test)

0.1834319526627219

In [ ]:
def get_intent_by_model(text):
    return log_reg.predict(vectorizer.transform([text]))[0]

In [ ]:
def bot(question):
  intent = get_intent_by_model(question)
  return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [ ]:
# question = ''
# while True:
#   question = input()
#   if question != 'стоп':
#       answer = bot(question)
#       print(answer)
#   else:
#       break

# День 3

In [ ]:
# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    question = update.message.text
    try:
      answer = bot(question)
    except:
      answer = 'Извините, что-то сломалось =('
      
    update.message.reply_text(answer)


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1971454798:AAHLLbwzKp8hXfHLNo_KHg23c7420dsbstc")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
main()

2021-09-15 18:33:53,006 - apscheduler.scheduler - INFO - Scheduler started
2021-09-15 18:45:04,520 - telegram.ext.updater - ERROR - Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
2021-09-15 18:45:04,523 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/updater.py", line 646, in _network_loop_retry
    if not action_cb():
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/updater.py", line 601, in polling_action_cb
    allowed_updates=allowed_updates,
  File "/usr/local/lib/python3.7/dist-packages/telegram/ext/extbot.py", line 228, in get_updates
    api_kwargs=api_kwargs,
  File "/usr/local/lib/python3.7/dist-packages/telegram/bot.py", line 130, in decorator
    result = func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/telegram/bot.py", line 2865,